***
# Modulo de generacion de analiticas y visualizaciones 





In [2]:
import sqlite3
import pandas as pd

***
## 01. Analiticas generales relacionadas con la tabla suministrada Municipios. 

El análisis de la información encontrada en la tabla de municipios evidencia la distribución de la población total y rural por departamento en Colombia, destacando la concentración poblacional según el análisis de Pareto. Los primeros cinco departamentos —Bogotá D.C., Antioquia, Valle del Cauca, Cundinamarca y Atlántico— concentran aproximadamente el 49% de la población total del país. Bogotá D.C. representa por sí sola cerca del 15% del total nacional, seguida por Antioquia (28%) y Valle del Cauca (37%). Esto demuestra una alta concentración de la población en pocas regiones del territorio nacional.

En relación con la población rural, Antioquia es el departamento con mayor cantidad de habitantes rurales (1.351.914), seguido por Cauca (1.014.835), Nariño (962.136) y Cundinamarca (852.271). Mientras tanto, departamentos como Bogotá D.C. presentan una proporción rural mínima respecto a su población total, a diferencia de regiones como Cauca y Nariño, donde el peso de la población rural es considerable. Estos hallazgos resaltan la coexistencia de grandes centros urbanos y zonas mayoritariamente rurales, aspectos clave para la formulación de políticas públicas y la asignación de recursos.


In [3]:
MUNICIPIOS =  pd.read_excel("02_Tablas_Normalizadas\\02_Municipios Normalizado.xlsx")
MUNICIPIOS['Poblacion_Rural'] = (MUNICIPIOS['PopTot'] * MUNICIPIOS['Rural'] / 100).astype(int)
MUNICIPIOS.head(2)

,Unnamed: 0,DP,Departamento,MPIO,Municipio,Superficie,PopTot,Rural,Region,Poblacion_Rural
0,0,5,Antioquia,5001,Medellín,350.666623,2634570,1.6,Región Eje Cafetero,42153
1,1,5,Antioquia,5002,Abejorral,497.566212,21622,56.7,Región Eje Cafetero,12259


In [4]:
# Se consolida la informacion relacionada con el departamento, total poblacion y total poblacion rural 
TAB_DEP = MUNICIPIOS.pivot_table(index=['DP', 'Departamento'],
                                 values=["PopTot", "Poblacion_Rural"],
                                 aggfunc="sum"
                                ).reset_index().sort_values("PopTot", ascending=False)

# Calcular porcentaje acumulado Pareto
TAB_DEP['% Pareto PopTot'] = TAB_DEP['PopTot'].cumsum() / TAB_DEP['PopTot'].sum() * 100

# Mostrar el resultado
TAB_DEP.head()

,DP,Departamento,Poblacion_Rural,PopTot,% Pareto PopTot
2,11,Bogotá D C,31751,7937898,14.974282
0,5,Antioquia,1351914,6951825,28.088407
23,76,Valle Del Cauca,657158,4652512,36.865040
10,25,Cundinamarca,852271,3657407,43.764479
1,8,Atlántico,141987,2845169,49.131689


In [7]:
import pandas as pd
import plotly.graph_objects as go
import nbformat

df = TAB_DEP.copy()

# Colores complementarios
color_pop = "#008080"      # Verde azulado (Poblacion Total)
color_rural = "#ff8000"    # Naranja fuerte (Poblacion Rural)
color_pareto = "#8000ff"   # Morado (Pareto)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Bar(x=df["Departamento"],
                        y=df["PopTot"],
                        name="Población Total",
                        marker_color=color_pop,
                        yaxis="y1"))

fig.add_trace(go.Bar(x=df["Departamento"],
                     y=df["Poblacion_Rural"],
                     name="Población Rural",
                     marker_color=color_rural,
                     yaxis="y1"))

fig.add_trace(go.Scatter(x=df["Departamento"],
                         y=df["% Pareto PopTot"],
                         mode="lines+markers",
                         name="% PopTot",
                         marker_color=color_pareto,
                         yaxis="y2"))

fig.update_layout(
    title='Pareto población por departamento',
    xaxis=dict(
        title='Departamento',
        tickangle=60,                # Gira más los nombres
        tickfont=dict(size=9),       # Reduce tamaño de fuente
    ),
    yaxis=dict(title='Población'),
    yaxis2=dict(title='% Acumulado', overlaying='y', side='right', range=[0, 100]),
    legend=dict(x=0.01, y=0.99),
    barmode='group',
    margin=dict(l=40, r=40, t=40, b=180),  # Más espacio abajo
    template='simple_white'
)
# Guardar como HTML
fig.write_html("03_Visualizaciones y Textos\\01_Departamentos_poblacion_pareto.html")


***
## 02. Analiticas relacionadas con los prestadores REPS

La presente revisión se enfocó en la capacidad instalada para la atención de salud mental en Colombia, utilizando la información contenida en la tabla de municipios y complementando con datos del archivo <span style="color:#006400;"><b>04_CapacidadInstalada.csv</b></span> del REPS vigente. Se analizaron las categorías **Salud Mental, Salud Mental Adulto y Salud Mental Pediátrico**, considerando principalmente la disponibilidad de camas, camillas y sillas asignadas para estos servicios. Esta información resulta fundamental para identificar la infraestructura disponible y posibles brechas en la oferta de servicios de salud mental a nivel nacional, contribuyendo a orientar la planeación sectorial y la toma de decisiones en salud pública, en línea con las prioridades identificadas por el Ministerio de Salud y Protección Social en la red de servicios del país.


In [11]:
# El código conecta a una base SQLite, carga toda la tabla 02_Servicios en un DataFrame pandas y cierra la conexión.

ruta_db = "02_Tablas_Normalizadas\\reps_database.db"
conn = sqlite3.connect(ruta_db)
df_capacidad = pd.read_sql_query('SELECT * FROM "04_CapacidadInstalada";', conn)
conn.close()

In [ ]:
# Se filtran los los items relacionados con salud mental 
SALUD_MENTAL =  ['Salud Mental',
                 'Salud Mental Adulto',
                 'Salud Mental Pediátrico']

df_capacidad["SALUD_MENTAL"] = df_capacidad['coca_nombre'].apply(lambda CAD: True if CAD in SALUD_MENTAL else False)

FILTRO_SALUDMENTAL = df_capacidad[df_capacidad["SALUD_MENTAL"]].pivot_table(index = ['DP'],
                                                                            values = 'cantidad',     # Camas, camillas o sillas               
                                                                            aggfunc= "sum").reset_index().sort_values("cantidad",
                                                                                                                       ascending=False)
FILTRO_SALUDMENTAL_MERG = FILTRO_SALUDMENTAL.merge(TAB_DEP, how = "inner", left_on = "DP", right_on="DP")
FILTRO_SALUDMENTAL_MERG.head(4)

,DP,cantidad,Departamento,Poblacion_Rural,PopTot,% Pareto PopTot
0,11,2294,Bogotá D C,31751,7937898,14.974282
1,13,812,Bolívar,612451,2267813,57.924386
2,8,805,Atlántico,141987,2845169,49.131689
3,5,645,Antioquia,1351914,6951825,28.088407


***
# 02_Capacidad Instalada para la Atención de Salud Mental en Colombia: Análisis por Categoría y Equipamiento

Este análisis evalúa la capacidad instalada para atención en salud mental en Colombia, utilizando datos del REPS sobre camas, camillas y sillas disponibles en servicios generales, adultos y pediátricos. Los resultados identifican brechas críticas para orientar la planificación sectorial.
La visualización muestra mediante círculos proporcionales la relación entre infraestructura disponible y población rural por departamento, evidenciando regiones prioritarias que requieren fortalecimiento inmediato.
El estudio revela severas inequidades territoriales: Chocó presenta apenas 16 camas para 352,000 habitantes rurales (0.05 camas/1000), Cauca dispone de 295 camas para un millón de habitantes rurales y Córdoba cuenta con 425 camas para 887,737 personas. Estas deficiencias contrastan con Bogotá, que concentra 2,294 camas para solo 31,751 habitantes rurales, confirmando una distribución de recursos que perjudica sistemáticamente a las comunidades rurales colombianas.
Gráfico 2. Relación entre Equipos servicios habilitados para Salud Mental y Población Rural por Departamento (Colombia) (grafico disponible HTML)


In [ ]:
import pandas as pd
import plotly.express as px

df = FILTRO_SALUDMENTAL_MERG
fig = px.scatter(df, 
                 x='Poblacion_Rural', 
                 y='cantidad', 
                 size='cantidad',
                 size_max=60,
                 color='Departamento',
                 hover_name='Departamento',
                 title='Capacidad de atención vs Población Rural por Departamento',
                 labels={'Poblacion_Rural':'Población Rural', 'cantidad':'Cantidad de Equipos'})

fig.update_layout(xaxis_title='Población Rural', yaxis_title='Cantidad de Camas, Camillas o Sillas')

# Guardar visualización en HTML
fig.write_html('03_Visualizaciones y Textos\\03_visualizacion_salud_mental.html')

